In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
import seaborn as sns
from datetime import datetime
from utils import (
    agregar_meses_segun_inflacion,
    obtener_escalar_de_inflacion,
    get_bias,
    set_bias,
    show_plot,
)

In [ ]:
last_check = datetime(
    2025, 2, 1
)  # ACTUALIZAR CUANDO SE AGREGUEN DATOS FUTUROS A ESTA FECHA Y REVISAR TODOS LOS DATOS
set_bias(last_check)

In [ ]:
get_bias()

# Análisis sobre datos de mi sueldo ingresados manualmente por mí

In [ ]:
datos_personales = pd.read_csv("../data/sueldos.csv")

In [ ]:
datos_personales["sueldo_bruto_usd_oficial"] = (
    datos_personales["sueldo_bruto_ars"] / datos_personales["dolar_oficial"]
)
datos_personales["sueldo_bruto_usd_blue"] = (
    datos_personales["sueldo_bruto_ars"] / datos_personales["dolar_blue"]
)
datos_personales["sueldo_neto_usd_oficial"] = (
    datos_personales["sueldo_neto_ars"] / datos_personales["dolar_oficial"]
)
datos_personales["sueldo_neto_usd_blue"] = (
    datos_personales["sueldo_neto_ars"] / datos_personales["dolar_blue"]
)
datos_personales.tail()

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    datos_personales["sueldo_bruto_ars"],
    label="Sueldo Bruto en ARS",
    color="#FFAAAA",
    linestyle=":",
    marker="o",
)
ax1.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    datos_personales["sueldo_neto_ars"],
    label="Sueldo Neto en ARS",
    color="#FF0000",
    linestyle="-",
    marker="o",
)
ax1.set_ylabel("Sueldo en ARS")
ax1.set_xlabel("Fecha")
ax1.set_title("Evolución Histórica del Sueldo Bruto y Neto")

ax2 = ax1.twinx()

ax2.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    datos_personales["sueldo_bruto_usd_oficial"],
    label="Sueldo Bruto en USD (Oficial)",
    color="#AAAAFF",
    linestyle=":",
    marker="o",
)
ax2.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    datos_personales["sueldo_bruto_usd_blue"],
    label="Sueldo Bruto en USD (Blue)",
    color="#AAFFAA",
    linestyle=":",
    marker="o",
)
ax2.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    datos_personales["sueldo_neto_usd_oficial"],
    label="Sueldo Neto en USD (Oficial)",
    color="#0000FF",
    linestyle="-",
    marker="o",
)
ax2.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    datos_personales["sueldo_neto_usd_blue"],
    label="Sueldo Neto en USD (Blue)",
    color="#00FF00",
    linestyle="-",
    marker="o",
)
ax2.set_ylabel("Sueldo en USD")

lines, labels, lines2, labels2 = (
    ax1.get_legend_handles_labels() + ax2.get_legend_handles_labels()
)
lines = lines + lines2
labels = labels + labels2
fig.legend(
    lines,
    labels,
    loc="upper left",
    bbox_to_anchor=(0.1, 1),
    bbox_transform=ax1.transAxes,
)

plt.xticks(rotation=45)
plt.grid()
plt.show()


In [ ]:
datos_personales["sueldo_ajustado"] = datos_personales["sueldo_neto_ars"].iloc[0]

for i in range(1, len(datos_personales)):
    if (
        datos_personales.loc[i, "puesto"] != datos_personales.loc[i - 1, "puesto"]
        or datos_personales.loc[i, "seniority"]
        != datos_personales.loc[i - 1, "seniority"]
    ):
        datos_personales.loc[i, "sueldo_ajustado"] = datos_personales.loc[
            i - 1, "sueldo_ajustado"
        ] + (
            datos_personales.loc[i, "sueldo_neto_ars"]
            - datos_personales.loc[i - 1, "sueldo_neto_ars"]
        )
    elif datos_personales.loc[i, "puesto"] != None:
        datos_personales.loc[i, "sueldo_ajustado"] = datos_personales.loc[
            i - 1, "sueldo_ajustado"
        ] * (1 + datos_personales.loc[i, "inflacion_del_mes_anterior"] / 100)
    else:
        datos_personales.loc[i, "sueldo_ajustado"] = 0.00
datos_personales["sueldo_ajustado"].tail()

In [ ]:
plot_id = plt.figure(figsize=(10, 6))
plt.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    datos_personales["sueldo_neto_ars"],
    label="Sueldo Neto",
    color="blue",
    marker="o",
)
plt.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    datos_personales["sueldo_ajustado"],
    label="Sueldo Ajustado por Inflación",
    color="red",
    linestyle="--",
    marker="x",
)

show_plot(
    plot_id,
    "Mes",
    "Sueldo (ARS)",
    "Comparación de Sueldo Neto vs Sueldo Ajustado por Inflación",
)

In [ ]:
avg_salary_blue = (
    datos_personales.groupby(["puesto", "seniority"])["sueldo_bruto_usd_blue"]
    .mean()
    .reset_index()
)
avg_salary_oficial = (
    datos_personales.groupby(["puesto", "seniority"])["sueldo_bruto_usd_oficial"]
    .mean()
    .reset_index()
)

plt.figure(figsize=(12, 8))

bar_width = 0.35
seniorities = avg_salary_blue["seniority"].unique()
indices = np.arange(len(seniorities))

for i, puesto in enumerate(avg_salary_blue["puesto"].unique()):
    group_blue = avg_salary_blue[avg_salary_blue["puesto"] == puesto]
    group_oficial = avg_salary_oficial[avg_salary_oficial["puesto"] == puesto]

    bar_positions = indices + i * bar_width

    plt.bar(
        bar_positions,
        group_blue["sueldo_bruto_usd_blue"],
        width=bar_width,
        label=f"{puesto} (Blue)",
        alpha=0.7,
        color="blue",
    )
    plt.bar(
        bar_positions + bar_width,
        group_oficial["sueldo_bruto_usd_oficial"],
        width=bar_width,
        label=f"{puesto} (Oficial)",
        alpha=0.7,
        color="orange",
    )

plt.title(
    "Comparación de Sueldo Bruto en USD (Blue vs Oficial) por Puesto y Seniority"
)
plt.xlabel("Seniority")
plt.ylabel("Sueldo Promedio en USD")
plt.xticks(indices + bar_width, seniorities, rotation=45)
plt.legend(title="Puesto y Tipo de Cambio")
plt.tight_layout()
plt.show()


# Análisis de datos de Bienes varios

In [ ]:
### USAR PARA ACTUALIZAR MANUALMENTE EL CSV DE VUELOS

# vuelos = pd.read_csv("../data/vuelos.csv")
# vuelos = agregar_meses_segun_inflacion(vuelos, 3, "precio_en_ars")
# vuelos

In [ ]:
vuelos = pd.read_csv("../data/vuelos.csv")
vuelos["fecha"] = pd.to_datetime(vuelos["fecha"])
vuelos.set_index("fecha", inplace=True)
monthly_vuelos = vuelos[["precio_en_ars"]].resample("M").mean()
monthly_vuelos["precio_en_ars"] = monthly_vuelos["precio_en_ars"].interpolate()
monthly_vuelos["destino"] = vuelos["destino"].resample("M").ffill()
monthly_vuelos.tail(5)

In [ ]:
monthly_vuelos = monthly_vuelos[-len(datos_personales) :]
plot_id = plt.figure(figsize=(10, 6))
plt.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    datos_personales["sueldo_neto_ars"],
    label="Sueldo Neto",
    color="blue",
    marker="o",
)
plt.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    monthly_vuelos["precio_en_ars"],
    label="Vuelos a EEUU",
    color="red",
    linestyle="--",
    marker="x",
)

y_min = min(
    datos_personales["sueldo_neto_ars"].min(), monthly_vuelos["precio_en_ars"].min()
)
y_max = max(
    datos_personales["sueldo_neto_ars"].max(), monthly_vuelos["precio_en_ars"].max()
)
yticks = np.linspace(y_min, y_max, num=10)

plt.yticks(yticks)

show_plot(
    plot_id,
    "Mes",
    "Monto en ARS",
    f"Comparación de Sueldo Neto vs Vuelos a EEUU (${monthly_vuelos.iloc[-1]['precio_en_ars']:.0f})",
)

In [ ]:
alquileres = pd.read_excel(
    "https://www.estadisticaciudad.gob.ar/eyc/wp-content/uploads/2024/11/MI_DAP_AX15.xlsx",
    header=2,
    skipfooter=3,
)
alquileres.columns = ["año", "mes", "precio_1_amb", "precio_2_amb", "precio_3_amb"]
del alquileres["precio_2_amb"]
del alquileres["precio_3_amb"]
alquileres["año"] = alquileres["año"].ffill()
alquileres["año"] = alquileres["año"].astype(int)
alquileres["precio_1_amb"] *= 0.7
alquileres = agregar_meses_segun_inflacion(alquileres, 1, "precio_1_amb", "mes")
alquileres = alquileres[-len(datos_personales) :]
alquileres.tail()

In [ ]:
plot_id = plt.figure(figsize=(10, 6))
plt.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    datos_personales["sueldo_neto_ars"],
    label="Sueldo Neto",
    color="blue",
    marker="o",
)
plt.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    alquileres["precio_1_amb"],
    label="Alquiler 1 Ambiente",
    color="red",
    linestyle="--",
    marker="x",
)

y_min = min(datos_personales["sueldo_neto_ars"].min(), alquileres["precio_1_amb"].min())
y_max = max(datos_personales["sueldo_neto_ars"].max(), alquileres["precio_1_amb"].max())
yticks = np.linspace(y_min, y_max, num=10)

plt.yticks(yticks)

show_plot(
    plot_id,
    "Mes",
    "Monto en ARS",
    f"Comparación de Sueldo Neto vs Alquileres de 1 Ambiente (${alquileres.iloc[-1]['precio_1_amb']:.0f})",
)

In [ ]:
autos = pd.read_csv("../data/autos.csv")
autos = agregar_meses_segun_inflacion(autos, -1, "ars_promedio", "fecha")
autos

In [ ]:
autos["ars_promedio"] = (autos["ars_promedio"] * 1.5) / 100
plot_id = plt.figure(figsize=(10, 6))
plt.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    datos_personales["sueldo_neto_ars"],
    label="Sueldo Neto",
    color="blue",
    marker="o",
)
plt.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    autos["ars_promedio"],
    label="Auto promedio en 100 cuotas",
    color="red",
    linestyle="--",
    marker="x",
)

show_plot(
    plot_id,
    "Mes",
    "Monto en ARS",
    f'Comparación de Sueldo Neto vs Auto Promedio en 100 cuotas (actualmente ${autos.iloc[-1]["ars_promedio"]*100:.0f})',
)

In [ ]:
big_mac = pd.read_csv(
    "https://github.com/TheEconomist/big-mac-data/releases/download/2024-07/big-mac-full-index.csv"
)
big_mac = big_mac[big_mac["iso_a3"] == "ARG"][["date", "local_price"]]
big_mac["date"] = pd.to_datetime(big_mac["date"])
big_mac.set_index("date", inplace=True)
big_mac = big_mac.resample("M").mean()
big_mac["local_price"] = big_mac["local_price"].interpolate()
big_mac = agregar_meses_segun_inflacion(big_mac, 7, "local_price")
big_mac = big_mac[-len(datos_personales) :]
big_mac["local_price"] = big_mac["local_price"] * 60
big_mac.tail(10)

In [ ]:
plot_id = plt.figure(figsize=(10, 6))
plt.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    datos_personales["sueldo_neto_ars"],
    label="Sueldo Neto",
    color="blue",
    marker="o",
)
plt.plot(
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str),
    big_mac["local_price"],
    label="Valor Big-Mac (x60)",
    color="red",
    linestyle="--",
    marker="x",
)

y_min = min(datos_personales["sueldo_neto_ars"].min(), big_mac["local_price"].min())
y_max = max(datos_personales["sueldo_neto_ars"].max(), big_mac["local_price"].max())
yticks = np.linspace(y_min, y_max, num=10)

plt.yticks(yticks)

show_plot(
    plot_id,
    "Mes",
    "Monto en ARS",
    f"Comparación de Sueldo Neto vs Valor Big-Mac (x60=${big_mac.iloc[-1]['local_price']:.0f})",
)

In [ ]:
variacion_sueldo_neto_ars = (
    datos_personales["sueldo_neto_ars"].iloc[-1] * 100
) / datos_personales["sueldo_neto_ars"].iloc[0]
variacion_alquileres = (alquileres["precio_1_amb"].iloc[-1] * 100) / alquileres[
    "precio_1_amb"
].iloc[0]
variacion_big_mac = (big_mac["local_price"].iloc[-1] * 100) / big_mac[
    "local_price"
].iloc[0]
variacion_autos = (autos["ars_promedio"].iloc[-1] * 100) / autos["ars_promedio"].iloc[0]
variacion_monthly_vuelos = (
    monthly_vuelos["precio_en_ars"].iloc[-1] * 100
) / monthly_vuelos["precio_en_ars"].iloc[0]
variacion_inflacion = 100
for index in range(1, len(datos_personales["inflacion_del_mes_anterior"])):
    variacion_inflacion *= 1 + (
        datos_personales["inflacion_del_mes_anterior"][index] / 100
    )
variaciones = [
    variacion_sueldo_neto_ars,
    variacion_inflacion,
    variacion_alquileres,
    variacion_big_mac,
    variacion_autos,
    variacion_monthly_vuelos,
]

etiquetas = [
    "Variación Mi Neto",
    "Inflación Acumulada",
    "Variación Alquileres",
    "Variación Big Mac",
    "Variación Autos",
    "Variación Vuelos",
]

combined = list(zip(variaciones, etiquetas))
combined.sort(key=lambda x: x[0], reverse=True)
variaciones, etiquetas = zip(*combined)

color = ["brown", "green", "orange", "red", "blue", "yellow"]

plot_id = plt.figure(figsize=(10, 6))
plt.bar(
    etiquetas,
    variaciones,
    color=[
        color[index] if value != "Variación Mi Neto" else "purple"
        for index, value in enumerate(etiquetas)
    ],
)
show_plot(
    plot_id,
    "Categorías",
    "Variación Porcentual (%)",
    "Variación Porcentual de Diferentes Bienes 11/2023 a Presente",
)

# Predicción de sueldo en los próximos 12 meses

In [ ]:
col_fecha = (
    datos_personales["año"].astype(str) + "-" + datos_personales["mes"].astype(str)
)
col_fecha = pd.to_datetime(col_fecha)
col_mes_ordinal = col_fecha.map(lambda date: date.toordinal())

In [ ]:
X = np.array(col_mes_ordinal).reshape(-1, 1)
y_ars = datos_personales["sueldo_bruto_ars"]
y_usd_oficial = datos_personales["sueldo_bruto_usd_oficial"]
y_usd_blue = datos_personales["sueldo_bruto_usd_blue"]

In [ ]:
model_ars = LinearRegression()
model_usd_oficial = LinearRegression()
model_usd_blue = LinearRegression()

model_ars.fit(X, y_ars)
model_usd_oficial.fit(X, y_usd_oficial)
model_usd_blue.fit(X, y_usd_blue)
pass

In [ ]:
last_date = col_fecha.max()
future_dates = [last_date + pd.DateOffset(months=i) for i in range(1, 13)]
future_dates_ordinal = np.array([date.toordinal() for date in future_dates]).reshape(
    -1, 1
)
future_dates

In [ ]:
pred_ars = model_ars.predict(future_dates_ordinal)
print("Predicciones en ARS:\n", pred_ars, "\n")
pred_usd_oficial = model_usd_oficial.predict(future_dates_ordinal)
print("Predicciones en USD Oficial:\n", pred_usd_oficial, "\n")
pred_usd_blue = model_usd_blue.predict(future_dates_ordinal)
print("Predicciones en USD Blue:\n", pred_usd_blue)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))

# ARS plot
ax1.plot(
    future_dates,
    pred_ars,
    label="Sueldo Bruto en ARS",
    color="#FF0000",
    linestyle="-",
    marker="o",
)
ax1.set_title("Proyección del Sueldo Bruto en ARS")
ax1.set_ylabel("Sueldo en ARS")
ax1.set_xlabel("Fecha")
ax1.legend()

# USD plot
ax2.plot(
    future_dates,
    pred_usd_oficial,
    label="Sueldo Bruto en USD (Oficial)",
    color="#0000FF",
    linestyle="-",
    marker="o",
)
ax2.plot(
    future_dates,
    pred_usd_blue,
    label="Sueldo Bruto en USD (Blue)",
    color="#00FF00",
    linestyle="-",
    marker="o",
)
ax2.set_title("Proyección del Sueldo Bruto en USD")
ax2.set_ylabel("Sueldo en USD")
ax2.set_xlabel("Fecha")
ax2.legend()

plt.xticks(rotation=45)
plt.grid()
plt.tight_layout()
plt.show()

# Análisis sobre datos de sueldos generales en Argentina

In [ ]:
rem_bruta_sec_priv = pd.read_csv(
    "https://infra.datos.gob.ar/catalog/sspm/dataset/153/distribution/153.1/download/remuneracion-bruta-asalariados-registrados-sector-privado-mensual-pesos-base-2009.csv"
)
rem_bruta_sec_priv.tail(3)

In [ ]:
datos_arg: list[dict[str, any]] = []
datos_arg.append(
    {
        "fecha": rem_bruta_sec_priv.iloc[-1]["indice_tiempo"],
        "valor": float(
            rem_bruta_sec_priv.iloc[-1]["asalariados_priv_remuneracion_mediana"]
        )
        * obtener_escalar_de_inflacion(3),
        "label": "Bruto Mediana",
    }
)
datos_arg.append(
    {
        "fecha": rem_bruta_sec_priv.iloc[-1]["indice_tiempo"],
        "valor": float(
            rem_bruta_sec_priv.iloc[-1]["asalariados_priv_remuneracion_promedio"]
        )
        * obtener_escalar_de_inflacion(3),
        "label": "Bruto Promedio",
    }
)

In [ ]:
distribucion_personal_ingreso_por_deciles_trismestrales = pd.read_csv(
    "https://infra.datos.gob.ar/catalog/sspm/dataset/301/distribution/301.1/download/distribucion-personal-ingreso-por-deciles-trismestrales.csv"
)
distribucion_personal_ingreso_por_deciles_trismestrales.tail(3)

In [ ]:
res = distribucion_personal_ingreso_por_deciles_trismestrales.iloc[-1][
    "decil_8_oip":"decil_10_oip"
]
datos_arg.extend(
    [
        {
            "fecha": distribucion_personal_ingreso_por_deciles_trismestrales[
                "indice_tiempo"
            ].iloc[-1],
            "valor": float(value) * obtener_escalar_de_inflacion(7),
            "label": index,
        }
        for index, value in res.items()
    ]
)

In [ ]:
remuneracion_imponible_promedio_trabajadores_estables_ripte = pd.read_csv(
    "https://infra.datos.gob.ar/catalog/sspm/dataset/158/distribution/158.1/download/remuneracion-imponible-promedio-trabajadores-estables-ripte-total-pais-pesos-serie-mensual.csv"
)
remuneracion_imponible_promedio_trabajadores_estables_ripte.tail(3)

In [ ]:
datos_arg.append(
    {
        "fecha": remuneracion_imponible_promedio_trabajadores_estables_ripte.iloc[-1][
            "indice_tiempo"
        ],
        "valor": float(
            remuneracion_imponible_promedio_trabajadores_estables_ripte.iloc[-1][
                "ripte"
            ]
        )
        * obtener_escalar_de_inflacion(2),
        "label": "RIPTE",
    }
)

In [ ]:
datos_arg.append(
    {
        "fecha": str(datos_personales["año"].iloc[-1])
        + "-"
        + str(datos_personales["mes"].iloc[-1])
        + "-01",
        "valor": float(datos_personales["sueldo_bruto_ars"].iloc[-1]),
        "label": "Mi Bruto",
    }
)
datos_arg.append(
    {
        "fecha": str(datos_personales["año"].iloc[-1])
        + "-"
        + str(datos_personales["mes"].iloc[-1])
        + "-01",
        "valor": float(datos_personales["sueldo_neto_ars"].iloc[-1]),
        "label": "Mi Neto",
    }
)
datos_arg.sort(key=lambda x: x["valor"])

In [ ]:
plot_id = plt.figure(figsize=(15, 6))
plt.bar(
    [dato["label"] for dato in datos_arg],
    [dato["valor"] for dato in datos_arg],
    width=0.9,
)
show_plot(
    plot_id,
    "Categorías",
    "Sueldo en Millones",
    "Comparación del Sueldo Actual con Datos del Gobierno",
)

# Análisis sobre datos de sueldos IT en Argentina

In [ ]:
sysarmy = pd.read_csv("../data/2025_1_sysarmy.csv")
sysarmy = sysarmy[
    [
        "dedicacion",
        "ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos",
        "seniority",
    ]
]
sysarmy["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"] = sysarmy[
    "ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"
] * obtener_escalar_de_inflacion(0)
sysarmy_fulltime = sysarmy[sysarmy["dedicacion"] == "Full-Time"]
promedios_seniority = sysarmy_fulltime.groupby("seniority")[
    "ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"
].mean()
sysarmy_fulltime = sysarmy[
    (sysarmy["dedicacion"] == "Full-Time")
    & (sysarmy["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"].notnull())
]
sueldo_actual = pd.DataFrame(
    {
        "dedicacion": "Full-Time",
        "ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos": datos_personales[
            "sueldo_neto_ars"
        ].iloc[-1],
        "seniority": "Sueldo Actual",
        "Color": "purple",
        "Ancho": 20,
    },
    index=[0],
)
sysarmy_fulltime = sysarmy_fulltime.sort_values(
    by="ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"
).reset_index(drop=True)

juniors = sysarmy_fulltime[sysarmy_fulltime["seniority"] == "Junior"]
semiseniors = sysarmy_fulltime[sysarmy_fulltime["seniority"] == "Semi-Senior"]
seniors = sysarmy_fulltime[sysarmy_fulltime["seniority"] == "Senior"]

juniors_90 = juniors[
    "ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"
].quantile(0.95)
juniors = juniors[
    juniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"] <= juniors_90
]

semiseniors_90 = semiseniors[
    "ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"
].quantile(0.95)
semiseniors_05 = semiseniors[
    "ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"
].quantile(0.05)
semiseniors = semiseniors[
    semiseniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"]
    <= semiseniors_90
]
semiseniors = semiseniors[
    semiseniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"]
    >= semiseniors_05
]

seniors_90 = seniors[
    "ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"
].quantile(0.95)
seniors_05 = seniors[
    "ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"
].quantile(0.05)
seniors = seniors[
    seniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"] <= seniors_90
]
seniors = seniors[
    seniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"] >= seniors_05
]

In [ ]:
juniors["Color"] = "green"
juniors["Ancho"] = 0.3
semiseniors["Color"] = "blue"
semiseniors["Ancho"] = 0.3
seniors["Color"] = "red"
seniors["Ancho"] = 0.3

sysarmy_ordenado = (
    pd.concat([juniors, semiseniors, seniors, sueldo_actual])
    .sort_values(by="ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos")
    .reset_index(drop=True)
)

unique_seniorities = sysarmy_ordenado["seniority"].unique()
palette = sns.color_palette("husl", len(unique_seniorities))
color_map = dict(zip(unique_seniorities, palette))

plot_id = plt.figure(figsize=(15, 8))
plt.bar(
    range(len(sysarmy_ordenado)),
    sysarmy_ordenado["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"],
    color=sysarmy_ordenado["Color"],
    width=sysarmy_ordenado["Ancho"],
)

plt.bar(0, 0, color="green", label="Junior")
plt.bar(0, 0, color="blue", label="Semi-Senior")
plt.bar(0, 0, color="red", label="Senior")
plt.bar(0, 0, color="purple", label="Sueldo Actual")

show_plot(
    plot_id,
    "Índice de cada salario (ordenado)",
    "Sueldo Neto (ARS)",
    "Distribución de Sueldos IT Full-Time Ordenados por Valor",
)

In [ ]:
summary = [
    sueldo_actual["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"][0],
    juniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"].mean(),
    juniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"].median(),
    semiseniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"].mean(),
    semiseniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"].median(),
    seniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"].mean(),
    seniors["ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos"].median(),
]

etiquetas = [
    "Mi Neto",
    "Media Jrs",
    "Mediana Jrs",
    "Media SSrs",
    "Mediana SSrs",
    "Media Srs",
    "Mediana Srs",
]

combined = list(zip(summary, etiquetas))
combined.sort(key=lambda x: x[0])
summary, etiquetas = zip(*combined)

color = ["#FDD", "#FBB", "#F99", "#F77", "#F55", "#F33", "#F11"]

plot_id = plt.figure(figsize=(10, 6))
plt.bar(
    etiquetas,
    summary,
    color=[
        color[index] if value != "Mi Neto" else "purple"
        for index, value in enumerate(etiquetas)
    ],
    width=0.9,
)

show_plot(
    plot_id,
    "Categorías",
    "Sueldo en Millones",
    "Comparación del Sueldo Actual Neto con la Media y Mediana de todos los Seniorities",
)